In [20]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error
import crypt
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'
DB_NAME = 'grupo8e2'

In [23]:
conn = None
cursor = None

table_name = "restaurant"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/restaurantes2.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            df.append(line.split(';'))
        # Insertar datos en la tabla
    
    
    exitos = 0
    fallos = 0
    for row in df:
        try:
            insert_query = """INSERT INTO Restaurant (nombre,vigente,estilo,repartomin,sucursal,direccion,telefono,area) 
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
            cursor.execute(insert_query, row)
            conn.commit()
            exitos +=1
        except(Exception, Error) as error:
            print(error)
            conn.rollback()
            fallos += 1


    conn.commit()
    print("Data loaded successfully in table {} for {} rows.".format(table_name, exitos))
    print("Data loaded unsuccessfully in table {} for {} rows.".format(table_name, fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Data loaded successfully in table restaurant for 171 rows.
Data loaded unsuccessfully in table restaurant for 0 rows.


In [22]:
conn = None
cursor = None

table_name = "plato"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/platos.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            df.append(line.split(';')[:-3])
        # Insertar datos en la tabla
    exitos = 0
    fallos = 0
    for row in df:
        try:
            insert_query = """INSERT INTO Plato (id,nombre,descripcion,disponibilidad,estilo,restriccion,ingredientes,porciones,precio,tiempo) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            cursor.execute(insert_query, row)
            conn.commit()
            exitos += 1
        except (Exception, Error) as error:
            if 'value too long for type character' in str(error):
                largo_columna = int(str(error).split(' ')[-1].replace('varying(', '').replace(')', ''))
                if len(row[1]) > largo_columna:
                    try:
                        conn.rollback()
                        cursor.execute("ALTER TABLE Plato ALTER COLUMN nombre TYPE VARCHAR(%s)", [len(row[1])])
                        cursor.execute(insert_query, row)
                        conn.commit()
                        exitos += 1
                    except (Exception, Error) as error:
                        print(error)
                        fallos += 1
                        conn.rollback()

                else:
                    fallos += 1
                    print(error)
                    conn.rollback()
            else:
                print(error)
                fallos += 1
                conn.rollback()

        conn.commit()

    print("Data loaded successfully in table {} for {} rows.".format(table_name, exitos))
    print("Data loaded unsuccessfully in table {} for {} rows.".format(table_name, fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(0) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(1) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(2) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(3) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(4) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(5) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(6) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(7) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(8) already exists.

duplicate key value violates unique constraint "plato_pkey"
DETAIL:  Key (id)=(9) already exists.

duplicate 

In [24]:
conn = None
cursor = None

table_name = "PlatoRestaurant"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    with open('../CSV/platos.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = line.split(';')
            df.append([row[0], row[-3]])

    cursor = conn.cursor()

    exitos = 0
    fallos = 0
    for row in df:
        try:
            insert_query = """WITH plato_cte AS (
                                SELECT p.id AS id
                                FROM plato as p
                                WHERE p.id = %s
                                ),
                                restaurant_cte AS (
                                SELECT r.id AS id
                                FROM restaurant as r
                                WHERE r.nombre = %s
                                )
                                INSERT INTO PlatoRestaurant (plato_id, restaurant_id)
                                SELECT plato_cte.id, restaurant_cte.id
                                FROM plato_cte, restaurant_cte;
                            """
            cursor.execute(insert_query, row)
            conn.commit()
            exitos += 1
        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
    conn.commit()
    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Data loaded successfully for 492 rows in table PlatoRestaurant.
Data loaded unsuccessfully for 0 rows


In [25]:
conn = None
cursor = None

table_name = "comuna"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/comuna2.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:

            df.append(line.split(';'))
        # Insertar datos en la tabla
    
    
    
    for row in df:
        try:
            insert_query = """INSERT INTO Comuna (cut,nombre,provincia,region) 
                                VALUES (%s, %s, %s, %s)"""
            cursor.execute(insert_query, row)
            conn.commit()
        except:
            conn.rollback()

    conn.commit()
    print("Data loaded successfully in table {}.".format(table_name))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Data loaded successfully in table comuna.


In [6]:
conn = None
cursor = None

table_name = "Usuario"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/clientes.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 6:
                row[3] = crypt.crypt(row[3])
                df.append(row[:-2])
            # Insertar datos en la tabla
    
    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """INSERT INTO Usuario (nombre,email,telefono,clave,tipo) 
                                VALUES (%s, %s, %s, %s, 'cliente')"""
            cursor.execute(insert_query, row)
            exitos += 1
            conn.commit()
        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()


    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Agustin.Matte@bbdduc.utopia) already exists.

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Agustin.Reyes@bbdduc.utopia) already exists.

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Alejandro.Benitez@bbdduc.utopia) already exists.

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Alejandro.Benitez@bbdduc.utopia) already exists.

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Alejandro.Benitez@bbdduc.utopia) already exists.

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Alejandro.Sierra@bbdduc.utopia) already exists.

duplicate key value violates unique constraint "usuario_email_key"
DETAIL:  Key (email)=(Alejandro.Sierra@bbdduc.utopia) already exists.

duplicate key value violates unique c

In [7]:
conn = None
cursor = None

table_name = "Direccion"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/clientes.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = line.split(';')[-2:]
            if len(row) == 2:
                direccion = [' '.join(row[0].split(',')[0].split()[:-1]), row[0].split(',')[0].split()[-1], row[1]]
                df.append(direccion)
    
    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """INSERT INTO Direccion (calle, numero, comuna_cut)
                                VALUES (%s, %s, %s)"""
            cursor.execute(insert_query, row)
            conn.commit()
            exitos += 1
        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()

    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

duplicate key value violates unique constraint "direccion_comuna_cut_calle_numero_key"
DETAIL:  Key (comuna_cut, calle, numero)=(5301, Calle Larga, 789) already exists.

duplicate key value violates unique constraint "direccion_comuna_cut_calle_numero_key"
DETAIL:  Key (comuna_cut, calle, numero)=(13114, Av. Apoquindo, 456) already exists.

duplicate key value violates unique constraint "direccion_comuna_cut_calle_numero_key"
DETAIL:  Key (comuna_cut, calle, numero)=(13114, Av. Apoquindo, 456) already exists.

duplicate key value violates unique constraint "direccion_comuna_cut_calle_numero_key"
DETAIL:  Key (comuna_cut, calle, numero)=(13114, Av. Apoquindo, 456) already exists.

duplicate key value violates unique constraint "direccion_comuna_cut_calle_numero_key"
DETAIL:  Key (comuna_cut, calle, numero)=(13101, Calle Estado, 456) already exists.

duplicate key value violates unique constraint "direccion_comuna_cut_calle_numero_key"
DETAIL:  Key (comuna_cut, calle, numero)=(13101, Cal

In [8]:
conn = None
cursor = None

table_name = "DireccionUsuario"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/clientes.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        cuts = []
        calles = []
        numeros = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 6:
                
                cliente = row[0]
                direccion = row[-2:]
                cuts = direccion[-1]
                calle = ' '.join(direccion[0].split(',')[0].split(' ')[:-1])
                numero = direccion[0].split(',')[0].split(' ')[-1]

                df.append([cliente, cuts, calle, numero])


    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """WITH cliente_cte AS (
                                SELECT id AS usuario_id
                                FROM usuario as u
                                WHERE u.nombre = %s
                                ),
                                direccion_cte AS (
                                    SELECT d.id AS direccion_id
                                    FROM direccion d
                                    WHERE d.comuna_cut = %s AND d.calle = %s AND d.numero = %s
                                )
                                INSERT INTO DireccionUsuario (usuario_id, direccion_id)
                                SELECT c.usuario_id, d.direccion_id
                                FROM cliente_cte c, direccion_cte d
                            """
            
            cursor.execute(insert_query, row)
            conn.commit()
            exitos += 1
        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()


    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Data loaded successfully for 299 rows in table DireccionUsuario.
Data loaded unsuccessfully for 0 rows


In [9]:
conn = None
cursor = None

table_name = "Empresa"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/cldeldes.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 13:
                df.append(row[4:-2])

            # Insertar datos en la tabla
    
    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """INSERT INTO Empresa (nombre, vigente, telefono, tiempo_despacho, precio_despacho, precio_mensual, precio_anual) 
                            VALUES (%s, %s, %s, %s, %s, %s, %s)"""
            cursor.execute(insert_query, row)      
            conn.commit()      
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()

    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56933034573) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56933034573) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56933034573) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56952133575) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56929246270) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56992140878) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56936853345) already exists.

duplicate key value violates unique constraint "empresa_telefono_key"
DETAIL:  Key (telefono)=(56952133575) already exists.



In [10]:
conn = None
cursor = None

table_name = "ConvenioEmpresaRestaurant"

try:
    # Conexión con la base de datos
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    # Abrir CSV
    with open('../CSV/pedidos2.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')        
        df = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 8:
                df.append([row[2], row[4]])
    
    

    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            row[1] = tuple(row[1].split())
            insert_query = """WITH empresa_cte AS (
                                SELECT id AS empresa_id
                                FROM empresa as e
                                WHERE e.nombre = %s
                               ),

                               restaurant_cte AS (
                                    SELECT pr.restaurant_id AS restaurant_id
                                    FROM platoRestaurant pr
                                    WHERE pr.plato_id IN %s 
                               )
                               INSERT INTO ConvenioEmpresaRestaurant (empresa_id, restaurant_id)
                               SELECT e.empresa_id, r.restaurant_id
                               FROM empresa_cte e, restaurant_cte r
                            """
            cursor.execute(insert_query, row)        
            conn.commit()    
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()

    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Data loaded successfully for 124 rows in table ConvenioEmpresaRestaurant.
Data loaded unsuccessfully for 0 rows


In [11]:
conn = None
cursor = None

table_name = "Despachador"

try:
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()
    
    with open('../CSV/cldeldes.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        
        df = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 13:
                df.append(row[-2:])
    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """INSERT INTO Despachador (nombre, telefono) 
                               VALUES (%s, %s)
                            """
            cursor.execute(insert_query, row)     
            conn.commit()       
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))

except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56108433226) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56038948627) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56475496149) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56765003948) already exists.



duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56093787321) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56921126580) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56787334517) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56787334517) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56395625093) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56477780729) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)=(56038948627) already exists.

duplicate key value violates unique constraint "despachador_telefono_key"
DETAIL:  Key (telefono)

In [12]:
conn = None
cursor = None

table_name = "Suscripcion"

try:
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()
    cursor.execute("SET DateStyle = 'ISO, DMY';")
    with open('../CSV/suscripciones.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        
        df = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 6:
                df.append(row)
    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """WITH usuario_cte AS (
                                SELECT u.id AS usuario_id
                                FROM usuario as u
                                WHERE u.email = %s
                                ),
                                empresa_cte AS (
                                SELECT e.id AS empresa_id
                                FROM empresa as e
                                WHERE e.nombre = %s
                                )

                               INSERT INTO Suscripcion (usuario_id, empresa_id, estado, ultimo_pago, fecha_ultimo_pago, ciclo)
                               SELECT usuario_cte.usuario_id, empresa_cte.empresa_id, %s, %s, %s, %s
                               FROM usuario_cte, empresa_cte;
                            """
            cursor.execute(insert_query, row)    
            conn.commit()        
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
        

    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))

except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Data loaded successfully for 409 rows in table Suscripcion.
Data loaded unsuccessfully for 0 rows


In [13]:
conn = None
cursor = None

table_name = "Pedido"

try:
    conn = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    with open('../CSV/pedidos2.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        header = data[0].split(';')
        df = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 8:
                df.append({h: r for h, r in zip(header, row)})
    exitos= 0
    fallos= 0
    for row in df:
        try:
            insert_query = """WITH usuario_cte AS (
                                SELECT u.id AS usuario_id
                                FROM usuario as u
                                WHERE u.email = %s
                                )

                               INSERT INTO Pedido (usuario_id, id, fecha, hora, estado)
                               SELECT usuario_cte.usuario_id, %s, %s, %s, TRIM(%s)
                               FROM usuario_cte;
                            """
            
            keys_to_extract = ['cliente', 'id','fecha', 'hora', 'estado']
            extracted = [row[key] for key in keys_to_extract]
            cursor.execute("SET DateStyle = 'ISO, DMY';")
            cursor.execute(insert_query, extracted)    
            conn.commit()
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
        
    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))

except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

try:
    conn = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    with open('../CSV/calificacion.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')
        cal = []
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 3:
                cal.append(row)
    exitos = 0
    fallos = 0
    for row in cal:
        try:
            update_query = """UPDATE Pedido SET evaluacion_cliente = %s, evaluacion_servicio = %s WHERE id = %s"""
            cursor.execute(update_query, row[::-1])
            conn.commit()
            exitos += 1
        except (Exception, Error) as error:
            print(error)
            fallos +=1 
            conn.rollback()
            
    print('Data Updated successfully for {} rows'.format(exitos))
    print('Data Updated unssuccesfully for {} rows'.format(fallos))
except (Exception, Error) as error:
    print("Error updating data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    

Data loaded successfully for 124 rows in table Pedido.
Data loaded unsuccessfully for 0 rows
Data Updated successfully for 93 rows
Data Updated unssuccesfully for 0 rows


In [14]:
conn = None
cursor = None

table_name = "Despacho"

try:
    conn = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = conn.cursor()

    with open('../CSV/pedidos2.csv', 'r', encoding = 'ISO-8859-1') as file:
        data = file.read().split('\n')

        df =[]
        for line in data[1:]:
            row = line.split(';')
            if len(row) == 8:
                df.append([row[0], row[2]])

    exitos = 0
    fallos = 0
    
    for row in df:
        try:
            insert_query = """WITH pedido_cte AS (
                                SELECT p.id AS pedido_id
                                FROM pedido as p
                                WHERE p.id = %s
                                ),
                                empresa_cte AS (
                                SELECT e.id AS empresa_id
                                FROM empresa as e
                                WHERE e.nombre = %s
                                )
                               INSERT INTO Despacho (pedido_id, empresa_id) 
                               SELECT pedido_id, empresa_id
                               FROM pedido_cte, empresa_cte;
                            """
            cursor.execute(insert_query, row)       
            conn.commit()     
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
            
    print('Dta loaded successfully for {} rows in table {}.'.format(exitos, table_name))
    print('Data loaded unsuccessfully for {} rows'.format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Dta loaded successfully for 124 rows in table Despacho.
Data loaded unsuccessfully for 0 rows


In [15]:

conn = None
cursor = None
table_name = "ServicioDespacho"

try:
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    cursor = conn.cursor()

    with open('../CSV/pedidos2.csv', 'r', encoding='ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = [s for s in line.split(';')]
            if len(row) == 8:
                id_list = [int(x) for x in row[4].split()] 
                df.append([row[0], id_list, row[3]])

    exitos = 0
    fallos = 0

    for row in df:
        try:
            insert_query = """
            WITH pedido_cte AS (
                SELECT p.id AS pedido_id
                FROM pedido AS p
                WHERE p.id = %s
            ),
            rest_cte AS (
                SELECT r.id AS restaurant_id
                FROM restaurant AS r JOIN platoRestaurant AS pr ON r.id = pr.restaurant_id
                WHERE pr.plato_id = ANY(%s)
            ),
            despachador_cte AS (
                SELECT d.id AS despachador_id
                FROM despachador AS d
                WHERE d.nombre = %s
            )
            INSERT INTO ServicioDespacho (pedido_id, despachador_id, restaurant_id)
            SELECT pedido_id, despachador_id, restaurant_id
            FROM pedido_cte, despachador_cte, rest_cte;
            """
            
            cursor.execute(insert_query, row) 
            conn.commit()
            exitos += 1

        except (Exception, Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, psycopg2.Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Data loaded successfully for 124 rows in table ServicioDespacho.
Data loaded unsuccessfully for 0 rows


In [16]:
conn = None
cursor = None

table_name = "ContenidoPedido"

try:
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    cursor = conn.cursor()

    with open('../CSV/pedidos2.csv', 'r', encoding='ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = [s for s in line.split(';')]
            if len(row) == 8:
                id_list = [int(x) for x in row[4].split()]
                df.append([row[0], id_list])
    

    exitos = 0
    fallos = 0

    for row in df:
        for pid in row[1]:
            try:
                insert_query = """INSERT INTO ContenidoPedido (pedido_id, plato_id)
                                    VALUES (%s, %s);
                                    """
                
                cursor.execute(insert_query, (row[0], pid))  # Passing the list of IDs directly
                conn.commit()
                exitos += 1

            except (Exception, psycopg2.Error) as error:
                print(error)
                fallos += 1
                conn.rollback()
    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

insert or update on table "contenidopedido" violates foreign key constraint "contenidopedido_pedido_id_fkey"
DETAIL:  Key (pedido_id)=(3) is not present in table "pedido".

insert or update on table "contenidopedido" violates foreign key constraint "contenidopedido_pedido_id_fkey"
DETAIL:  Key (pedido_id)=(3) is not present in table "pedido".

insert or update on table "contenidopedido" violates foreign key constraint "contenidopedido_pedido_id_fkey"
DETAIL:  Key (pedido_id)=(4) is not present in table "pedido".

insert or update on table "contenidopedido" violates foreign key constraint "contenidopedido_pedido_id_fkey"
DETAIL:  Key (pedido_id)=(18) is not present in table "pedido".

insert or update on table "contenidopedido" violates foreign key constraint "contenidopedido_pedido_id_fkey"
DETAIL:  Key (pedido_id)=(79) is not present in table "pedido".

insert or update on table "contenidopedido" violates foreign key constraint "contenidopedido_pedido_id_fkey"
DETAIL:  Key (pedido_id)

In [17]:
conn = None
cursor = None

table_name = "DespachadorEmpresa"


try:
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME
    )
    cursor = conn.cursor()

    with open('../CSV/pedidos2.csv', 'r', encoding='ISO-8859-1') as file:
        data = file.read().split('\n')
        df = []
        for line in data[1:]:
            row = [s for s in line.split(';')]
            if len(row) == 8:
                df.append([row[2], row[3]])
    

    exitos = 0
    fallos = 0

    for row in df:
        try:
            insert_query = """WITH empresa_cte AS (
                                SELECT e.id AS empresa_id
                                FROM empresa AS e
                                WHERE e.nombre = %s
                                ),
                                despachador_cte AS (
                                SELECT d.id AS despachador_id
                                FROM despachador AS d
                                WHERE d.nombre = %s
                                )
                                
                                INSERT INTO DespachadorEmpresa (despachador_id, empresa_id)
                                SELECT despachador_id, empresa_id
                                FROM despachador_cte, empresa_cte;
                                """
            
            cursor.execute(insert_query, row)  # Passing the list of IDs directly
            exitos += 1
            conn.commit()
        except (Exception, psycopg2.Error) as error:
            print(error)
            fallos += 1
            conn.rollback()
        
    print("Data loaded successfully for {} rows in table {}.".format(exitos, table_name))
    print("Data loaded unsuccessfully for {} rows".format(fallos))
except (Exception, Error) as error:
    print("Error loading data:", error)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Data loaded successfully for 124 rows in table DespachadorEmpresa.
Data loaded unsuccessfully for 0 rows


In [18]:
# reset serial

# # Conexión con la base de datos
# conn = psycopg2.connect(
#     user=DB_USER,
#     password=DB_PASSWORD,
#     host=DB_HOST,
#     port=DB_PORT,
#     dbname=DB_NAME
# )
# # Crear un cursor para ejecutar consultas
# cursor = conn.cursor()
# cursor.execute("ALTER SEQUENCE usuario_id_seq RESTART WITH 1")
# conn.commit()
# cursor.close()
# conn.close()

In [26]:
with open('../CSV/comuna2.csv', 'r',encoding= 'mac_roman') as file:
    data = file.read().split('\n')
    for line in data:
        print(line)

    

cut;'nombre';'provincia';'region'
15201;'Putre';'Parinacota';'Arica y Parinacota'
13605;'Peñaflor';'Talagante';'Metropolitana de Santiago'
13604;'Padre Hurtado';'Talagante';'Metropolitana de Santiago'
15202;'General Lagos';'Parinacota';'Arica y Parinacota'
1101;'Iquique';'Iquique';'Tarapacá'
1107;'Alto Hospicio';'Iquique';'Tarapacá'
1401;'Pozo Almonte';'Tamarugal';'Tarapacá'
1402;'Camiña';'Tamarugal';'Tarapacá'
1403;'Colchane';'Tamarugal';'Tarapacá'
1404;'Huara';'Tamarugal';'Tarapacá'
1405;'Pica';'Tamarugal';'Tarapacá'
2101;'Antofagasta';'Antofagasta';'Antofagasta'
2102;'Mejillones';'Antofagasta';'Antofagasta'
2103;'Sierra Gorda';'Antofagasta';'Antofagasta'
2104;'Taltal';'Antofagasta';'Antofagasta'
2201;'Calama';'El Loa';'Antofagasta'
2202;'Ollagüe';'El Loa';'Antofagasta'
2203;'San Pedro de Atacama';'El Loa';'Antofagasta'
2301;'Tocopilla';'Tocopilla';'Antofagasta'
2302;'María Elena';'Tocopilla';'Antofagasta'
3101;'Copiapó';'Copiapó';'Atacama'
3102;'Caldera';'Copiapó';'Atacama'
3103;'Ti

In [37]:
with open('../CSV/restaurantes2.csv', 'r',encoding= 'mac_roman') as file:
    data = file.read().split('\n')  # .replace('√©', 'é').replace('√°', 'á')
    for line in data:
        print(line)

nombre;vigente;estilo;repartomin;sucursal;direccion;telefono;area
DCCRestaurant;TRUE;fast food;10000;Las Parcelas;Av. Las Parcelas 1234, Macul, Santiago;562994152387; Macul
Meet Burger;FALSE;fast food;5000;Providencia;Av. Providencia 456, Las Condes, Santiago;56871243659; Las Condes
Meet Burger;FALSE;fast food;5000;Vitacura;Av. Vitacura 789, Vitacura, Santiago;56274568231; Vitacura
Meet Burger;FALSE;fast food;5000;Apoquindo;Av. Apoquindo 123, Las Condes, Santiago;56321078546; Las Condes
Meet Burger;FALSE;fast food;5000;Kennedy;Av. Kennedy 456, Las Condes, Santiago;562109876452; Las Condes
Burger Queen;FALSE;fast food;8000;Padre Mariano;Calle Padre Mariano 789, La Reina, Santiago;56784561032; La Reina
Burger Queen;FALSE;fast food;8000;LarraÔøΩn;Av. LarraÔøΩn 123, La Reina, Santiago;562367891254; La Reina
Delicia China;FALSE;internacional;11000;Recoleta;Av. Recoleta 456, Huechuraba, Santiago;56215647893; Huechuraba
Delicia China;FALSE;internacional;11000;San Pablo;Calle San Pablo 789, La